# Spanish Voice Cloning with Tortoise TTS

<a href="https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:

In [ ]:
# Check version and setup
import requests

def get_latest_commit():
    repo_api = "https://api.github.com/repos/juanvolpe/voiceJuan/commits/main"
    response = requests.get(repo_api)
    if response.status_code == 200:
        return response.json()['sha']
    return None

# Clone repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Check version
latest_commit = get_latest_commit()
if latest_commit:
    print(f"\nLatest GitHub commit: {latest_commit}")
    print("\nIf you want the latest version:")
    print("1. File → Reset runtime")
    print("2. Runtime → Run all")

# Install dependencies
!pip install -r requirements.txt